In [2]:
import cv2
import numpy as np
import math
from sort import Sort

model_config = 'src/yolov3.cfg'
model_weights = 'src/yolov3.weights'

class ObjectDetector():
    def __init__(self):
        net = cv2.dnn.readNet(model_config,model_weights)
        self.model = cv2.dnn_DetectionModel(net)
        self.model.setInputParams(size=(416,416),scale=1/255)
        self.classes_allowed = [0]
        
    def detect(self,img):
        bboxes = []
        class_ids,scores,boxes = self.model.detect(img,nmsThreshold=0.4)
        for class_id, score,box in zip(class_ids,scores,boxes):
            if score < 0.3:
                continue
            
            if class_id in self.classes_allowed:
                x,y,w,h = [a for a in box]
                box = [x,y,x+w,y+h,score]
                bboxes.append(np.array(box).astype('int64'))
        return np.array(bboxes)
    
def check(x1,y1,x2,y2,width,height):
    if x1<=0 or y1<=0 or x2>=width or y2>=height:
        return False
    else:
        return True

In [17]:
roi = np.array([[[637,717],
                [1055,573],
                [1261,673],
                [1192,709]]],np.int32)

In [18]:
def draw_roi(pts,color,frame):
    alpha = 0.95
    #create overlay
    overlay = np.zeros_like(frame,np.uint8)
    #draw
    cv2.fillPoly(overlay,pts,color)
    mask = overlay.copy().astype(bool)
    frame[mask] = cv2.addWeighted(frame, alpha, overlay, 1 - alpha, 0)[mask]
    cv2.polylines(frame,roi,True,(122,34,25),1)
    return frame

In [ ]:
detector = ObjectDetector()
mot = Sort()

cap = cv2.VideoCapture('src/test.mp4')
width  = int(cap.get(3)) # float `width`
height = int(cap.get(4))
tracked = []
people_count= []
if not cap.isOpened():
    print('there is a error openning the video file')
else:
    ret,frame = cap.read()
    cv2.imwrite('file.jpg',frame)
    while cap.isOpened():
        ret,frame = cap.read()
        if ret :
            bboxes = detector.detect(frame)
            tracked = mot.update(bboxes)
            for person in tracked:
                x1,y1,x2,y2,id = [int(a) for a in person]
                #checking if person crossed roi
                c = [(x1+x2)//2,(y1+y2)//2]
                crossed = cv2.pointPolygonTest(roi,c,False)
                if crossed==1 and id not in people_count and check(x1,y1,x2,y2,width,height):
                    people_count.append(id)
                print(people_count)
                cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,0),2)
                cv2.putText(frame,str(id),(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,1,(0,124,255),2)
                cv2.putText(frame,f'people count:{len(people_count)}',(30,45),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),3)
                frame = draw_roi(roi,(135,0,255),frame)
            cv2.imshow('video',frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else :
            break
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
        
                
            

In [ ]:
detector = ObjectDetector()
trackers = cv2.legacy.MultiTracker_create()
cap = cv2.VideoCapture('src/test.mp4')
ret,frame = cap.read()
width  = cap.get(3)  # float `width`
height = cap.get(4)
boxes = detector.detect(frame)
for box in boxes:
    tracker_j = cv2.legacy.TrackerCSRT_create()
    trackers.add(tracker_j,frame,box)

persons = {}
id = 0
initial = True
if not cap.isOpened():
    print('there is a error openning the video file')
else:
    while cap.isOpened():
        ret,frame = cap.read()
        if ret :
            (success,boxes) = trackers.update(frame)
            # intializing persons
            if initial:
                for box in boxes:
                    persons[id] = box
                    id+=1
                initial =False
            
            for (box,(id,person)) in zip(boxes,persons.copy().items()):
                (x,y,w,h) = person
                if not initial:
                    persons[id] = box
                # if some object goes beyond limits of frame
                if not check(x,y,w,h,height,width):
                    persons.pop(id)
                    del trackers
                    trackers = cv2.legacy.MultiTracker_create()
                    new_bbs = detector.detect(frame)
                    for box in new_bbs:
                        same = False
                        #checks if bbox is the same 
                        for key,value in persons.copy().items():
                            x1,y1,w1,h1 = box
                            x2,y2,w2,h2 = value
                            c1 = [x1+w1//2,y1+h1//2]
                            c2 = [x2+w2//2,y2+h2//2]
                            if math.dist(c1,c2)<5:
                                same = True
                                persons[key] = box
                        if same:
                            tracker = cv2.legacy.TrackerCSRT_create()
                            trackers.add(tracker,frame,box)
                        # if new object, track it
                        else:
                            x,y,w,h = box
                            if check(x,y,w,h,height,width):
                                id+=1
                                persons[id] = box
                                tracker = cv2.legacy.TrackerCSRT_create()
                                trackers.add(tracker,frame,box)
            print(persons)
            for id,person in persons.items():
                (x,y,w,h) = [int(a) for a in person]
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,12,255),2)
                cv2.putText(frame,str(id),(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            cv2.imshow('video',frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else :
            break
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    

In [10]:
import copy
import cv2
import numpy as np
capture = cv2.VideoCapture('src/test.mp4')
background_subtractor = cv2.bgsegm.createBackgroundSubtractorMOG()
length = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))



first_iteration_indicator = 1
for i in range(0, length):

    ret, frame = capture.read()

    # If first frame
    if first_iteration_indicator == 1:

        first_frame = copy.deepcopy(frame)
        height, width = frame.shape[:2]
        accum_image = np.zeros((height, width), np.uint8)
        
        first_iteration_indicator = 0
    else:
        filter = background_subtractor.apply(frame)  # remove the background

        threshold = 2
        maxValue = 2
        ret, th1 = cv2.threshold(filter, threshold, maxValue, cv2.THRESH_BINARY)

        # add to the accumulated image
        accum_image = cv2.add(accum_image, th1)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        color_image = cv2.applyColorMap(accum_image, cv2.COLORMAP_JET)
        result_overlay = cv2.addWeighted(frame, 0.7, color_image, 0.7, 0)
        cv2.imshow('video',result_overlay)
cv2.imwrite('result.jpg',result_overlay)

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1